### Install packages.

In [ ]:
%pip install -r requirements.txt

### Please input your OpenAI API key.

To get OpenAI API Key go to https://platform.openai.com/playground and after logging into your account, click on 'View API keys'.

**Note that the system cannot be used without entering your credit card information to your OpenAI account since the system interactions exceed the free API limitations. A typical conversation costs < $0.02.**

Your API key will only be accessible locally to this jupyter notebook and will not be stored or shared with others.

In [ ]:
from getpass import getpass
global openai_api_key
openai_api_key = getpass("Enter your OpenAI API key:")

### Run the conversational recommendation system.

In [ ]:
import gradio as gr
import yaml
import time
from conv_rec_system import ConvRecSystem
from domain_specific.classes.restaurants.geocoding.nominatim_wrapper import NominatimWrapper
from domain_specific.classes.restaurants.location_constraint_merger import LocationConstraintMerger
from domain_specific.classes.restaurants.location_status import LocationStatus
from domain_specific.classes.restaurants.location_filter import LocationFilter
from information_retriever.filter.word_in_filter import WordInFilter

with open("system_config.yaml") as f:
    system_config = yaml.load(f, Loader=yaml.FullLoader)
system_config['PATH_TO_DOMAIN_CONFIGS'] = f'domain_specific/configs/restaurant_configs'

with open(f"{system_config['PATH_TO_DOMAIN_CONFIGS']}/domain_specific_config.yaml") as f:
    domain_specific_config = yaml.load(f, Loader=yaml.FullLoader)
            
    geocoder = NominatimWrapper(location_bias=domain_specific_config.get("LOCATION_BIAS"))

    if geocoder.geocode("edmonton") is None:
        user_filter_objects = [WordInFilter(["location"], "address")]

        conv_rec_system = ConvRecSystem(
            system_config, openai_api_key,
            user_defined_filter=user_filter_objects, user_interface_str="demo")
    else:
        user_constraint_merger_objects = [LocationConstraintMerger(geocoder)]
        user_constraint_status_objects = [LocationStatus(geocoder)]
        user_filter_objects = [LocationFilter("location", ["latitude", "longitude"], 3, geocoder)]

        conv_rec_system = ConvRecSystem(
            system_config, openai_api_key, user_defined_constraint_mergers=user_constraint_merger_objects,
            user_constraint_status_objects=user_constraint_status_objects,
            user_defined_filter=user_filter_objects, user_interface_str="demo")

with open("chatbot.css", "r", encoding="utf-8") as f:
    chatbot_css = f.read()

with gr.Blocks(css=chatbot_css) as demo:
    gr.Markdown("# LLM Convrec")
    history = gr.State([[None, conv_rec_system.init_msg]])
    with gr.Row():
        chatbot = gr.Chatbot(
            value=[[None, conv_rec_system.init_msg]], show_label=False, elem_id="llm_conv_rec")
    with gr.Row(equal_height=True):
        with gr.Column(scale=8):
            user_input = gr.Textbox(show_label=False, placeholder="Enter text", container=False)
        with gr.Column(min_width=70, scale=1):
            send_button = gr.Button(value="Send")
        with gr.Column(min_width=70, scale=3):
            new_conv_button = gr.Button(value="New Conversation")

    def display_user_input(user_message, chatbot, history) -> tuple[gr.Textbox, gr.Chatbot, gr.State]:
        """
        Display user input.

        :param user_message: user input
        :param chatbot: chatbot that display chat history
        :param history: chat history
        :return: a tuple of textbox,chatbot, and state that are updated
        """
        chatbot[-1][1] = history[-1][1]
        return "", chatbot + [[user_message, None]], history + [[user_message, None]]

    def display_recommender_response(chatbot, history) -> tuple[gr.Chatbot, gr.State]:
        """
        Display recommender's response.

        :param chatbot: chatbot that display chat history
        :param history: chat history
        :return: a tuple of chatbot and state that are updated
        """
        bot_message = conv_rec_system.get_response(chatbot[-1][0])
        history[-1][1] = bot_message
        chatbot[-1][1] = ""
        for character in bot_message:
            chatbot[-1][1] += character
            time.sleep(0.015)
            yield chatbot, history

    def reset_state() -> tuple[gr.Textbox, gr.Chatbot, gr.State]:
        """
        Reset state.

        :return: a tuple of textbox, chatbot, and state that are reset
        """
        conv_rec_system.dialogue_manager.state_manager.reset_state()
        return "", [[None, conv_rec_system.init_msg]], [[None, conv_rec_system.init_msg]]

    user_input.submit(
        fn=display_user_input, inputs=[user_input, chatbot, history],
        outputs=[user_input, chatbot, history], queue=True).then(
            fn=display_recommender_response, inputs=[chatbot, history], outputs=[chatbot, history])

    send_button.click(
        fn=display_user_input, inputs=[user_input, chatbot, history],
        outputs=[user_input, chatbot, history], queue=True).then(
            fn=display_recommender_response, inputs=[chatbot, history], outputs=[chatbot, history])

    new_conv_button.click(fn=reset_state, outputs=[user_input, chatbot, history], queue=True)


if __name__ == "__main__":
    demo.queue()
    demo.launch()